# MLflow AI Gateway: Stay at the Frontier

## The Challenge

When new models (GPT-5, Claude Sonnet 4.5, Gemini 2.5) are released, platform teams need to **evaluate, compare, and gradually migrate** their apps — balancing **quality, latency, cost, and governance** — without breaking production or rewriting code.

---

## What You'll Learn

**Part 1: Fundamentals** (5 min)
1. ✅ Automated tracing - every call logged automatically
2. 📝 Prompt versioning - register and retrieve versions

**Part 2: Model Migration Workflow** (15 min)
3. 🔄 Baseline capture from current model
4. 🎯 Auto-optimize prompts for new model
5. 📊 Compare quality, latency, cost
6. 🚀 Gradual migration with A/B testing

**Key Benefit:** Inline APIs - just call functions, automatic MLflow logging!

## Setup

In [ ]:
import mlflowlite as mlflow
import os

# Set your API key
os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# Set experiment (all automatic runs go here)
mlflow.set_experiment("ai_gateway_demo")

print("✅ Setup complete")

---

# Part 1: Fundamentals

## 📊 Feature 1: Automatic Tracing

### The Old Way (Without Tracing)

You call an LLM:

```python
response = openai.chat.completions.create(...)
print(response)
```

Questions you can't answer:
- ❓ How much did that cost?
- ❓ How long did it take?
- ❓ Was the response quality good?
- ❓ Can I compare this to yesterday's version?

**You're flying blind!** 🛩️💨

---

### The New Way (With mlflowlite)

Same code, **automatic insights**:

In [ ]:
# Simple LLM call - automatically creates MLflow run!
@mlflow.trace
def classify_sentiment(text: str) -> str:
    """Classify sentiment - each call creates an MLflow run automatically"""
    response = mlflow.llm_call(
        model="openai/gpt-4",
        messages=[{"role": "user", "content": f"Classify sentiment as positive/negative/neutral: {text}"}],
        temperature=0
    )
    return response['choices'][0]['message']['content'].lower().strip()

# Just call it - automatic run creation!
result = classify_sentiment("This product is amazing!")

print(f"Result: {result}")
print("\n" + "="*60)
print("🎯 Value Unlocked: See Everything Automatically")
print("="*60)
print("✅ Cost tracked automatically")
print("✅ Latency tracked automatically")
print("✅ Tokens tracked automatically")
print("✅ MLflow run created automatically")
print("✅ Trace viewable in MLflow UI")
print("\n💡 Check MLflow UI: http://localhost:5000 → Traces")

### 🎯 What You Get for FREE

Every call automatically logs:
- 💰 **Cost** - Exact cost per call
- ⏱️ **Latency** - Response time
- 🎫 **Tokens** - Input + output tokens
- 📊 **Run ID** - Unique identifier
- 🔗 **UI Links** - Direct links to MLflow UI

**No manual logging. No boilerplate. Just insights!**

## 📝 Feature 2: Prompt Versioning

### The Old Way (Without Versioning)

**Monday:** You write a prompt. It works great!

**Tuesday:** You "improve" it. Now it's slower and costs more.

**Wednesday:** You want Monday's version back but... 😱 **You didn't save it!**

Questions you can't answer:
- ❓ Which version was cheaper?
- ❓ Which version was faster?
- ❓ What exactly did I change?
- ❓ Can I roll back?

**You're guessing in the dark!** 🎲

---

### The New Way (With Prompt Versioning)

Track every version automatically. Compare with real numbers.

In [ ]:
# Register prompt Version 1 (Simple)
prompt_v1 = mlflow.register_prompt(
    name="sentiment_classifier",
    template="Classify sentiment: {{text}}"
)

print(f"✅ Registered prompt 'sentiment_classifier' version 1")
print(f"   URI: {prompt_v1['uri']}")
print(f"   Template: {prompt_v1['template']}")
print("\n💡 View in MLflow UI: Prompts tab → sentiment_classifier")

In [ ]:
# Use Version 1
@mlflow.trace
def classify_v1(text: str) -> str:
    """Using prompt v1 - automatic run creation!"""
    prompt_text = prompt_v1['template'].replace('{{text}}', text)
    response = mlflow.llm_call(
        model="openai/gpt-4",
        messages=[{"role": "user", "content": prompt_text}],
        temperature=0,
        prompt_name="sentiment_classifier"  # Links trace to prompt version
    )
    return response['choices'][0]['message']['content']

# Test v1 - creates run automatically!
result_v1 = classify_v1("The service was excellent!")
print(f"v1 Result: {result_v1}")
print("\n✅ MLflow run created automatically with prompt linkage!")

In [ ]:
# Register Version 2 (Improved with explicit instructions)
prompt_v2 = mlflow.register_prompt(
    name="sentiment_classifier",
    template="""Classify the sentiment of the following text.

Text: {{text}}

Rules:
- Answer ONLY with: positive, negative, or neutral
- Use lowercase
- No explanation

Answer:"""
)

print(f"✅ Registered prompt 'sentiment_classifier' version 2")
print(f"   URI: {prompt_v2['uri']}")
print("\n📊 Now you have 2 versions tracked in MLflow!")

In [ ]:
# Use Version 2
@mlflow.trace
def classify_v2(text: str) -> str:
    """Using prompt v2 - automatic run creation!"""
    prompt_text = prompt_v2['template'].replace('{{text}}', text)
    response = mlflow.llm_call(
        model="openai/gpt-4",
        messages=[{"role": "user", "content": prompt_text}],
        temperature=0,
        prompt_name="sentiment_classifier"
    )
    return response['choices'][0]['message']['content']

# Test v2 - creates run automatically!
result_v2 = classify_v2("The service was excellent!")
print(f"v2 Result: {result_v2}")
print("\n✅ Another MLflow run created automatically!")
print("\n💡 Compare both runs in MLflow UI to see which is better!")

In [ ]:
# Retrieve specific versions
retrieved_v1 = mlflow.get_prompt(name="sentiment_classifier", version=1)
retrieved_v2 = mlflow.get_prompt(name="sentiment_classifier", version=2)

print("📚 Version History:\n")
print(f"Version 1: {retrieved_v1['template'][:50]}...")
print(f"Version 2: {retrieved_v2['template'][:50]}...")
print("\n✅ Git-like versioning for prompts!")
print("   • Roll back anytime")
print("   • Compare versions")
print("   • Track all changes")

---

# Part 2: Model Migration Workflow

## 🎯 The Scenario

You have a **production sentiment classifier** using GPT-4.

**GPT-4o-mini is released**: Faster and 50x cheaper!

**The Question**: Can you migrate without breaking production?

**The Answer**: Yes! Using inline APIs with automatic tracking.

---

## Step 1: Your Production App

In [ ]:
# Production prompt
prod_prompt = mlflow.register_prompt(
    name="production_sentiment",
    template="Classify sentiment as positive, negative, or neutral: {{text}}"
)

# Production function (GPT-4)
@mlflow.trace
def production_classifier(text: str, model: str = "openai/gpt-4") -> str:
    """Production classifier - automatic run creation!"""
    prompt_text = prod_prompt['template'].replace('{{text}}', text)
    response = mlflow.llm_call(
        model=model,
        messages=[{"role": "user", "content": prompt_text}],
        temperature=0,
        prompt_name="production_sentiment"
    )
    return response['choices'][0]['message']['content'].lower().strip()

# Test current production
result = production_classifier("Amazing product!")
print(f"GPT-4 Result: {result}")
print("\n✅ Current production: GPT-4")
print("   • MLflow run created automatically")
print("   • Cost/latency tracked automatically")

## Step 2: Collect Baseline Data

Collect outputs from GPT-4 on representative data. **Each call creates an MLflow run automatically!**

In [ ]:
# Production test cases
test_cases = [
    "This movie was absolutely fantastic!",
    "The service was terrible.",
    "It was okay, nothing special.",
    "I'm so disappointed with this purchase.",
    "Best experience ever!",
    "The product works as described.",
    "I can't believe how amazing this is!",
    "Worst customer support ever.",
    "It's fine for the price.",
    "This exceeded all my expectations!"
]

# Collect baseline - each call creates an MLflow run!
print("🔄 Collecting baseline from GPT-4...\n")
baseline_outputs = []

for i, text in enumerate(test_cases, 1):
    result = production_classifier(text, model="openai/gpt-4")  # Auto-creates run!
    baseline_outputs.append(result)
    print(f"{i}. {text[:35]:35s} → {result}")

print(f"\n✅ {len(baseline_outputs)} baseline outputs collected")
print(f"✅ {len(baseline_outputs)} MLflow runs created automatically!")
print("\n💡 All runs visible in MLflow UI with cost/latency data")

## Step 3: Test New Model

Test GPT-4o-mini with the **same prompt**. Spot the differences!

In [ ]:
# Test new model - each call creates an MLflow run!
print("🔄 Testing GPT-4o-mini with same prompt...\n")
new_model_outputs = []

for i, text in enumerate(test_cases, 1):
    result = production_classifier(text, model="openai/gpt-4o-mini")  # Auto-creates run!
    new_model_outputs.append(result)
    baseline = baseline_outputs[i-1]
    match = "✅" if result == baseline else "❌"
    print(f"{i}. {match} new: {result:8s} (baseline: {baseline})")

# Calculate consistency
matches = sum(1 for n, b in zip(new_model_outputs, baseline_outputs) if n == b)
consistency = matches / len(baseline_outputs) * 100

print(f"\n📊 Consistency: {consistency:.0f}%")
print(f"✅ {len(new_model_outputs)} more MLflow runs created automatically!")
if consistency < 90:
    print("\n⚠️  Need optimization!")

## Step 4: Auto-Optimize Prompt

Use DSPy to optimize the prompt for GPT-4o-mini.

In [ ]:
from mlflowlite.optimization import optimize_prompts

print("🎯 Optimizing prompt for GPT-4o-mini...\n")

# Training data
training_data = [
    {"text": text, "expected": baseline}
    for text, baseline in zip(test_cases[:5], baseline_outputs[:5])
]

# Optimize - this creates MLflow runs automatically during optimization!
optimized_text = optimize_prompts(
    task="sentiment",
    model="openai/gpt-4o-mini",
    training_data=training_data,
    original_prompt=prod_prompt['template']
)

print("✅ Optimization complete!")
print(f"\n📝 Original:\n{prod_prompt['template']}")
print(f"\n📝 Optimized:\n{optimized_text}")
print("\n✅ Optimization runs tracked in MLflow automatically!")

In [ ]:
# Register optimized prompt
opt_prompt = mlflow.register_prompt(
    name="production_sentiment",
    template=optimized_text
)

print(f"✅ Registered optimized prompt (new version)")
print(f"   URI: {opt_prompt['uri']}")

## Step 5: Evaluate Optimized Version

In [ ]:
# Optimized classifier
@mlflow.trace
def optimized_classifier(text: str) -> str:
    """Optimized classifier - automatic run creation!"""
    prompt_text = opt_prompt['template'].replace('{{text}}', text)
    response = mlflow.llm_call(
        model="openai/gpt-4o-mini",
        messages=[{"role": "user", "content": prompt_text}],
        temperature=0,
        prompt_name="production_sentiment"
    )
    return response['choices'][0]['message']['content'].lower().strip()

# Test optimized - each call creates an MLflow run!
print("🔄 Testing optimized version...\n")
optimized_outputs = []

for i, text in enumerate(test_cases, 1):
    result = optimized_classifier(text)  # Auto-creates run!
    optimized_outputs.append(result)
    baseline = baseline_outputs[i-1]
    match = "✅" if result == baseline else "❌"
    print(f"{i}. {match} optimized: {result:8s} (baseline: {baseline})")

# Calculate improvement
matches = sum(1 for o, b in zip(optimized_outputs, baseline_outputs) if o == b)
improved = matches / len(baseline_outputs) * 100

print(f"\n📊 Before: {consistency:.0f}% | After: {improved:.0f}%")
print(f"🎉 Improvement: +{improved - consistency:.0f} points!")
print(f"\n✅ {len(optimized_outputs)} more MLflow runs created automatically!")

## Step 6: Performance Comparison

In [ ]:
import pandas as pd

comparison = pd.DataFrame([
    {
        "Model": "GPT-4",
        "Prompt": "Original",
        "Consistency": "100%",
        "Latency": "~800ms",
        "Cost/1M": "$30",
        "Status": "🟢 Production"
    },
    {
        "Model": "GPT-4o-mini",
        "Prompt": "Original",
        "Consistency": f"{consistency:.0f}%",
        "Latency": "~300ms",
        "Cost/1M": "$0.60",
        "Status": "❌ Not Ready"
    },
    {
        "Model": "GPT-4o-mini",
        "Prompt": "Optimized",
        "Consistency": f"{improved:.0f}%",
        "Latency": "~300ms",
        "Cost/1M": "$0.60",
        "Status": "✅ Ready!"
    }
])

print("\n📊 Model Comparison:\n")
print(comparison.to_string(index=False))

print("\n💡 Benefits:")
print("   • 2.5x faster")
print("   • 50x cheaper")
print("   • Same quality")
print("   • All tracked in MLflow automatically!")

## Step 7: Gradual Migration (A/B Testing)

Roll out gradually with automatic tracking.

In [ ]:
import random

# Gradual rollout function
@mlflow.trace
def production_v2(text: str, rollout_pct: int = 20) -> dict:
    """
    Production with gradual rollout - automatic run creation!
    
    Args:
        text: Input text
        rollout_pct: % traffic to new model (0-100)
    """
    use_new = random.random() * 100 < rollout_pct
    
    if use_new:
        result = optimized_classifier(text)  # Auto-creates run!
        model = "gpt-4o-mini"
    else:
        result = production_classifier(text)  # Auto-creates run!
        model = "gpt-4"
    
    return {"sentiment": result, "model": model}

print("✅ Gradual rollout ready!")
print("   Every call creates MLflow run automatically")
print("   Track cost/latency per model in real-time")

In [ ]:
# Simulate 20% rollout
print("🚀 Simulating 20% rollout...\n")

stats = {"gpt-4": 0, "gpt-4o-mini": 0}

for i, text in enumerate(test_cases, 1):
    result = production_v2(text, rollout_pct=20)  # Auto-creates run!
    stats[result["model"]] += 1
    print(f"{i}. {result['model']:12s} | {result['sentiment']}")

print(f"\n📊 Distribution:")
for model, count in stats.items():
    print(f"   {model}: {count}/{len(test_cases)} ({count*10}%)")

print(f"\n✅ {len(test_cases)} MLflow runs created automatically!")
print("\n💡 Migration path:")
print("   5% → 20% → 50% → 80% → 100%")
print("   Monitor MLflow UI at each step")

## Step 8: Full Migration

In [ ]:
# Production V3: 100% new model
@mlflow.trace
def production_v3(text: str) -> str:
    """Production V3 - fully migrated - automatic run creation!"""
    return optimized_classifier(text)

# Test final version
print("🎉 Production V3 - 100% GPT-4o-mini\n")

samples = ["Amazing!", "Terrible.", "It's okay."]

for text in samples:
    result = production_v3(text)  # Auto-creates run!
    print(f"'{text}' → {result}")

print("\n✅ Migration complete!")
print("\n📈 Achieved:")
print("   • 2.5x faster")
print("   • 50x cheaper")
print("   • Same quality")
print("   • Zero downtime")
print("   • All automatically tracked in MLflow!")

---

# 🎯 Summary: What You Achieved

## Part 1: Fundamentals ✅

### 1. Automatic Tracing
- ✅ Every `@mlflow.trace` call creates MLflow run automatically
- ✅ Cost, latency, tokens tracked automatically
- ✅ No manual `start_run()` needed
- ✅ All traces visible in MLflow UI

### 2. Prompt Versioning
- ✅ Register prompts with `register_prompt()`
- ✅ Retrieve any version with `get_prompt()`
- ✅ Compare versions in MLflow UI
- ✅ Roll back anytime

## Part 2: Model Migration ✅

You migrated from **GPT-4 → GPT-4o-mini** with:

1. **Baseline Capture** - 10 runs created automatically
2. **New Model Test** - 10 more runs created automatically
3. **Optimization** - Optimization runs tracked automatically
4. **Evaluation** - 10 more runs created automatically
5. **A/B Testing** - Traffic split tracked automatically
6. **Full Migration** - 100% rollout with automatic tracking

**Total:** ~40+ MLflow runs created **automatically** with zero manual logging!

---

## 🔑 Key Takeaway

### Inline API = Zero Boilerplate

```python
# ❌ OLD WAY: Manual logging
with mlflow.start_run():
    result = call_llm()
    mlflow.log_metric("cost", cost)
    mlflow.log_metric("latency", latency)
    # ... more manual logging

# ✅ NEW WAY: Automatic logging
@mlflow.trace
def my_function():
    return mlflow.llm_call(...)  # Everything automatic!
```

**Result:** More insights, less code, always at the frontier! 🚀

---

## 📚 Resources

- [MLflow Auto-rewrite Prompts](https://mlflow.org/docs/latest/genai/prompt-registry/rewrite-prompts/)
- [MLflow Evaluation Guide](https://mlflow.org/docs/latest/genai/eval-monitor/quickstart/)
- [Prompt Management](https://mlflow.org/docs/latest/genai/prompt-registry/create-edit-prompts/)